In [ ]:
import os
import json
import csv

import re

import  ftfy

from findJsonKeys import FindJsonKey 
from populateFlatByJson import populate 


# transform comment file

In [13]:
facebook_comment_dir=os.environ("FACE_DATA")+"your_activity_across_facebook/comments_and_reactions/"



findKeys=FindJsonKey()
def transform(msg:dict,d:dict)-> dict:
    result={}
    result["title"]=ftfy.fix_text(msg["title"].replace("\n"," "))


    populate(d)

    result["timestamp"] = msg["timestamp"]
    return result

with open(os.path.join(os.path.dirname(facebook_comment_dir),"comment.csv"), 'w', newline='', encoding='utf-8') as csvFile:
    csvSave = csv.DictWriter(csvFile, fieldnames="title,author,timestamp,comment,media,external_context".split(","))
    csvSave.writeheader()
    
    for root,folders,files in os.walk(facebook_comment_dir):
        folder=root.replace(facebook_comment_dir,"")
        for file in [file for file in files if "comments" in file]:
            with open(os.path.join(root,file), 'r') as user_file:
                parsed_json = json.load(user_file)

                parsed_json = parsed_json["comments_v2"]
                attachments = [transform(msg,d) for msg in parsed_json if "attachments" in msg for a in msg["attachments"] for d in a['data']]
                comments =  [transform(msg,d["comment"]) for msg in parsed_json if "data" in msg for d in msg["data"]]
                
                csvSave.writerows(attachments)
                csvSave.writerows(comments)
                

print(findKeys.keys())                

# Transform Like File
Read all like files and stract the reaction person date

In [ ]:


findKeys=FindJsonKey()
def transform(msg,data):
    
    result =  {
        "title":ftfy.fix_text(msg["title"]),
        'reaction':data["reaction"]["reaction"],
        'timestamp':msg['timestamp']     
    }

    itens=re.findall("^Mariano Aloi ([^']+)'*s* (.*).$",ftfy.fix_text(msg["title"]))
    if len(itens) > 0 :
        itens=itens[0]
        result["typeItem"]=itens[1]  
        
        result["person"]=itens[0].replace( \
            "liked " if re.match("^liked .*",itens[0]) \
                else \
            "reacted to " if re.match("^reacted to .*",itens[0]) \
                else "","");
                             
    return result;
with open(os.path.join(os.path.dirname(facebook_comment_dir),"likes.csv"), 'w', newline='', encoding='utf-8') as csvFile:
    csvSave = csv.DictWriter(csvFile, fieldnames="title,timestamp,reaction,typeItem,person".split(","))
    csvSave.writeheader()
    for root,folders,files in os.walk(facebook_comment_dir):
        folder=root.replace(facebook_comment_dir,"")
        for file in [file for file in files if "likes_and_reactions" in file]:
            with open(os.path.join(root,file), 'r') as user_file:
                parsed_json = json.load(user_file)
                # findKeys.makeDecision(parsed_json)
                parsed_json = parsed_json = [transform(msg,data) for msg in parsed_json for data in msg["data"]]
                csvSave.writerows(parsed_json)
print(findKeys.keys())